In [ ]:
import pyproj

import numpy as np
import matplotlib.pyplot as plt
from shapely import geometry, ops
from datetime import datetime, timedelta

from sentinelhub import SHConfig, BBox, CRS, SentinelHubRequest, DataCollection, MimeType, bbox_to_dimensions, SentinelHubCatalog, MosaickingOrder

CLIENT_ID = ''
CLIENT_SECRET = ''

config = SHConfig()

if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET

In [ ]:
region = {"type":"Polygon","coordinates":[[[-66.481018,-8.885072],[-65.220337,-8.885072],[-65.220337,-10.052698],[-66.481018,-10.052698],[-66.481018,-8.885072]]]}
region = geometry.shape(region)

proj = pyproj.Transformer.from_crs(3857, 4326, always_xy=True)
proj_inv = pyproj.Transformer.from_crs(4326, 3857, always_xy=True)
region_wm = ops.transform(proj_inv.transform, region)


In [ ]:
n_years = 0

date_end = datetime(2023, 5, 7)
date_start = datetime(2023, 5, 7) - timedelta(days=n_years*365)
date = date_start
print(date.strftime("%Y-%m-%d"))

In [ ]:
print((date).strftime("%Y-%m-%d"))

In [ ]:
resolution = 100
bbox = BBox(bbox=region.bounds, crs=CRS.WGS84)
size = bbox_to_dimensions(bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {size} pixels")



In [ ]:
# def get_s2_l2a(s2_bbox, s2_date, config):
    # """ author: Alessandro """
evalscript_grd = """//VERSION=3
    function setup() {
      return {
        input: [{bands:["VV", "VH"]}],
        output: {
                    bands: 2,
                    sampleType: "FLOAT32"
                }
      };
    }

    function evaluatePixel(samples) {
      return [samples.VV,
              samples.VH];
    }
"""



In [ ]:
request2 = SentinelHubRequest(
    evalscript=evalscript_grd,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL1_IW,
            time_interval=[(date-timedelta(days=1)).strftime("%Y-%m-%d"),(date+timedelta(days=9)).strftime("%Y-%m-%d")],
            mosaicking_order = MosaickingOrder.MOST_RECENT,
            other_args={"processing": {"orthorectify": True, "backCoeff": "GAMMA0_TERRAIN"}},
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.TIFF),
        #SentinelHubRequest.output_response("userdata", MimeType.JSON),
    ],
    bbox=bbox,
    size=bbox_to_dimensions(bbox, resolution),
    config=config
)


In [ ]:
MosaickingOrder.LEAST_RECENT

In [ ]:
request2.payload

In [ ]:
image = request2.get_data()[0]

In [ ]:
image.shape

In [ ]:
np.nanmax(image), np.nanmin(image)

In [ ]:
plt.imshow(image[:,:,0])

In [ ]:
import rasterio
def save_tif(filename, image, poly, channels, crs, dtype='uint8', factor=1):
    height, width = image.shape[1:]
    geotiff_transform = rasterio.transform.from_bounds(poly.bounds[0], poly.bounds[1],
                                                       poly.bounds[2], poly.bounds[3],
                                                       width/factor, height/factor)

    new_dataset = rasterio.open(filename, 'w', driver='GTiff',
                                height=height/factor, width=width/factor,
                                count=channels, dtype=dtype,
                                crs=crs,
                                transform=geotiff_transform)

    # Write bands
    for i in range(channels):
        new_dataset.write(image[i],i+1)
    new_dataset.close()
    
    return True

In [ ]:
save_tif('./outputs_s1/'+str(date.year)+'.tif', np.nan_to_num(np.moveaxis(image,-1,0)), region_wm, channels=2, crs=3857,dtype='float32')
